In [1]:
import glob
import nbformat
from nbformat.v4 import new_code_cell
from nbconvert import HTMLExporter
from nbconvert.preprocessors import ExecutePreprocessor, CellExecutionError
import re
import textwrap
import shutil
import json
import pandas as pd
from pathlib import Path
from datetime import datetime


In [3]:
fpaths = glob.glob('../submissions/case-study-04/*.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths[:5]

['../submissions/case-study-04\\angeloalvino_10490_6521576_case_study_04_rideshare_trips.ipynb',
 '../submissions/case-study-04\\badilloguadalupe_40209_6559837_case_study_04_rideshare_trips.ipynb',
 '../submissions/case-study-04\\bahenadenise_31344_6558259_case_study_04_rideshare_trips.ipynb',
 '../submissions/case-study-04\\battaarnav_7441_6557146_case_study_04_rideshare_trips.ipynb',
 '../submissions/case-study-04\\fukagairikuto_337180_6508239_case_study_04_rideshare_trips.ipynb']

In [2]:
fpaths = glob.glob('test-notebooks/PCard-20230203-test.ipynb')
fpaths = list(filter(lambda f: not f.endswith('-graded.ipynb'), fpaths))
fpaths

['test-notebooks/PCard-20230203-test.ipynb']

In [3]:
from lambdagrader import *

graded_results = []

for notebook_path in fpaths:
    try:
        print('=============================')
        nb = nbformat.read(notebook_path, as_version=4)
        
        for cell in nb.cells:
            test_case_metadata = extract_test_case_metadata_from_cell(cell.source)

            if test_case_metadata:
                cell.source = convert_to_grader_code(cell.source)

            converted_notebook_path = notebook_path.replace('.ipynb', '-graded.ipynb')
            graded_notebook_filename = Path(converted_notebook_path).name

        add_scripts_to_notebook(nb)

        with open(converted_notebook_path, mode='w', encoding='utf-8') as f:
            nbformat.write(nb, f)

        nb = nbformat.read(converted_notebook_path, as_version=4)
        
        ep = ExecutePreprocessor(
            timeout=600,
            kernel_name='python3',
            allow_errors=True
        )
        ep.preprocess(nb)
        print(f'Running {converted_notebook_path} successful')
        
        graded_result_json_path = notebook_path.replace('.ipynb', '-result.json')
        shutil.move('lambdagrader-result.json', graded_result_json_path)
        print(f'Stored graded result as JSON to {graded_result_json_path}')
        
        with open(graded_result_json_path, mode='r') as f:
            graded_result = json.load(f)
            
        summary = ''
        summary += f"File: {graded_notebook_filename}\n"
        summary += f"Score: {graded_result['learner_score']} out of {graded_result['total_available']}\n"
        summary += f"Passed {graded_result['num_passed_cases']} out of {graded_result['num_test_cases']} test cases\n"
        summary += f"Grading took {graded_result['grading_duration_in_seconds']} seconds\n\n"
        summary += 'Test Case Summary\n'
        
        for o in graded_result['results']:
            summary += f"{o['test_case_name']} {'passed' if o['pass'] else 'failed'}: {o['points']} out of {o['available_points']} points\n"
            
            if not o['pass']:
                summary += f"[Autograder Output]\n{o['message']}\n\n"
                
        result_summary = {
            'filename': graded_notebook_filename,
            'grading_finished_at': graded_result['grading_finished_at'],
            'grading_duration_in_seconds': graded_result['grading_duration_in_seconds'],
            'learner_score': graded_result['learner_score'],
            'total_available': graded_result['total_available'],
            'num_test_cases': graded_result['num_test_cases'],
            'num_passed_cases': graded_result['num_passed_cases'],
            'num_failed_cases': graded_result['num_failed_cases'],
            'summary': summary
        }
        
        print(result_summary)
        graded_results.append(result_summary)
    except CellExecutionError as e:
        print(f'CellExecutionError on {notebook_path}')
        print('-----------------------------')
        print(e)
        
df_summary = pd.DataFrame(graded_results)

df_summary.to_csv(
    f"graded_result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
    index=None
)

Running test-notebooks/PCard-20230203-test-graded.ipynb successful
Stored graded result as JSON to test-notebooks/PCard-20230203-test-result.json
{'filename': 'PCard-20230203-test-graded.ipynb', 'grading_finished_at': '2023-02-03T08:07:23.393955', 'grading_duration_in_seconds': 21, 'learner_score': 82, 'total_available': 82, 'num_test_cases': 20, 'num_passed_cases': 20, 'num_failed_cases': 0, 'summary': 'File: PCard-20230203-test-graded.ipynb\nScore: 82 out of 82\nPassed 20 out of 20 test cases\nGrading took 21 seconds\n\nTest Case Summary\n1B: Create a list of column names passed: 2 out of 2 points\n1C: Read the CSV files into Pandas DataFrames passed: 3 out of 3 points\n1D: Filter only relevant data passed: 2 out of 2 points\n1E: Concatenate all transactions passed: 2 out of 2 points\n1F: Convert different date string formats into datetime types passed: 4 out of 4 points\n1G: Convert transaction_date and posted_date columns to datetime types passed: 3 out of 3 points\n1H: Convert amo

In [5]:
from lambdagrader import *

tcs = extract_test_cases_metadata_from_notebook( '../notebooks\\case-study-04-rideshare-trips-SOLUTION.ipynb')

s = 0

for o in tcs:
    s += o['points']
    
print(s)

80.0
